In [2]:
import pandas as pd
import numpy as np

# Importação e Cópia
Estou importando os dados diretamente do Gsheet, e na célula seguinte fazendo a cópia dos dataframes. A cópia serve para não ter o trabalho de importar dataframe da nuvem novamente.

In [3]:
DF_COMPRAS_2024 = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8/export?format=xlsx&id=117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8",
    sheet_name="Lançamento de NF Não Recorrente",
    usecols=[
        "Marca",
        "Unidade",
        "Item",
        "Valor Corrigido",
        "Mês Corrigido",
    ]
)

DF_DE_PARA_OFC = pd.read_excel(
    'https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=format=xlsx&id=1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ', 
    sheet_name="Unidades",
    usecols=[
        "Chave 2024",
        "Marca 2024",
        "Unidade 2024"
    ]
)

c:\Users\fernando.luiz\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Cell E1954 is marked as a date but the serial value 6692727.0 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


In [4]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024.copy()
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC.copy()

# Correção do Mês
Aqui estou covertendo os meses faltantes pelo mês atual, e não só isso, convertendo os valores para inteiro.

In [5]:
from datetime import datetime

DF_COMPRAS_2024_COPY["Mês Corrigido"] = pd.to_numeric(DF_COMPRAS_2024_COPY["Mês Corrigido"], "coerce")
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].fillna(0)
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].astype("int")
DF_COMPRAS_2024_COPY.loc[DF_COMPRAS_2024_COPY["Mês Corrigido"] == 0, "Mês Corrigido"] = datetime.now().month

# Correção de Marca e Unidade
Agora, vamos atualizar a marca e unidade para os nomes de 2024, nesse processo estou fazendo:
- Retirandos os dados faltantes
- Aplicando função ```stip()``` na Marca e Unidade, para retirar os espaços em branco
- Criando uma coluna apenas com a Unidade
- Criando a Chave 2024
- Aplicando o DePara e puxando as Marcas e Unidades atualizadas

In [6]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Marca"].notna()]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Unidade"].notna()]

DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC_COPY[DF_DE_PARA_OFC_COPY["Chave 2024"].notna()]
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC_COPY.drop_duplicates()

DF_COMPRAS_2024_COPY["Marca"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Marca"].strip(), axis=1)
DF_COMPRAS_2024_COPY["Unidade"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Unidade"].strip(), axis=1)

DF_COMPRAS_2024_COPY["Unidade Atualizada"] = DF_COMPRAS_2024_COPY.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_COMPRAS_2024_COPY["Chave 2024"] = DF_COMPRAS_2024_COPY["Marca"] + DF_COMPRAS_2024_COPY["Unidade Atualizada"] 

DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "NUs")]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "Visita BE")]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "Salta")]

DF_COMPRAS_2024_DE_PARA = pd.merge(DF_COMPRAS_2024_COPY, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

# Dividindo o DataFrame
Nesse momento estou divindo meu DataFrame em dois, um com os erros, no qual vou tratar e unir novamente ao dataset original.

In [7]:
DF_ERROS = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].isna()]
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].notna()]

In [8]:
DF_ERROS["Marca"] = DF_ERROS.apply(lambda x: x["Unidade"].split(" ")[0], axis=1)

DF_ERROS["Unidade Atualizada"] = DF_ERROS.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_ERROS["Chave 2024"] = DF_ERROS["Marca"] + DF_ERROS["Unidade Atualizada"]
DF_ERROS.drop(columns=[
    "Marca 2024",
    "Unidade 2024",
], inplace=True)

DF_ERROS = pd.merge(DF_ERROS, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

# Unindo as Bases de Dados

In [9]:
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido",
    "Item",
]]

DF_COMPRAS_2024_DE_PARA["Ano"] = 2024
DF_COMPRAS_2024_DE_PARA["NO"] = "Ativos"
DF_COMPRAS_2024_DE_PARA["Categoria"] = "Supply"
DF_COMPRAS_2024_DE_PARA["Tipo Valor"] = "Capex"
DF_COMPRAS_2024_DE_PARA["Tipo"] = "Real"

DF_ERROS_ATUALIZADOS = DF_ERROS[DF_ERROS["Marca 2024"].notna()]
DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido",
    "Item",
]]

DF_ERROS_ATUALIZADOS["Ano"] = 2024
DF_ERROS_ATUALIZADOS["NO"] = "Ativos"
DF_ERROS_ATUALIZADOS["Categoria"] = "Supply"
DF_ERROS_ATUALIZADOS["Tipo Valor"] = "Capex"
DF_ERROS_ATUALIZADOS["Tipo"] = "Real"

DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
    "Item"
]]

DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
    "Item", 
]]

In [10]:
DF_ATIVOS = pd.concat([DF_COMPRAS_2024_DE_PARA, DF_ERROS_ATUALIZADOS], axis=0)
DF_ATIVOS["Valor Corrigido"].sum() 

10767630.23

In [11]:
DF_ATIVOS["Item"] = DF_ATIVOS["Item"].str.lower().str.strip()

In [13]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [14]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fernando.luiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fernando.luiz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
DF_ATIVOS = DF_ATIVOS[DF_ATIVOS["Item"].notna()]

In [22]:
DF_ATIVOS["Token Item"] = DF_ATIVOS["Item"].apply(word_tokenize) 

C:\Users\fernando.luiz\AppData\Local\Temp\ipykernel_3300\939247097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_ATIVOS["Token Item"] = DF_ATIVOS["Item"].apply(word_tokenize)


In [27]:
list_stopwords = set(stopwords.words("portuguese"))
DF_ATIVOS["Token Item"] = DF_ATIVOS["Token Item"].apply(lambda x: [word for word in x if word not in list_stopwords])

C:\Users\fernando.luiz\AppData\Local\Temp\ipykernel_3300\2926019025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_ATIVOS["Token Item"] = DF_ATIVOS["Token Item"].apply(lambda x: [word for word in x if word not in list_stopwords])


In [30]:
DF_ATIVOS["Token Item"] = DF_ATIVOS["Token Item"].apply(lambda x: [word for word in x if word.isalpha()])


C:\Users\fernando.luiz\AppData\Local\Temp\ipykernel_3300\2112285845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_ATIVOS["Token Item"] = DF_ATIVOS["Token Item"].apply(lambda x: [word for word in x if word.isalpha()])


In [35]:
list_all_itens = []

for lista in DF_ATIVOS["Token Item"]:
    list_all_itens += lista

In [43]:
df_item = pd.DataFrame(list_all_itens)
df_item.columns = ["Item"]
df_item["Item"].value_counts()

Item
mat          257
móveis       233
esportivo    230
mobílias     226
mesa         157
            ... 
rodrigo        1
chedid         1
pinote         1
fracote        1
kits           1
Name: count, Length: 1707, dtype: int64

: 

In [32]:
DF_ATIVOS

,Valor Corrigido,Marca 2024,Unidade 2024,Ano,NO,Categoria,Tipo Valor,Tipo,Mês Corrigido,Item,Token Item
0,106020.0,Anglo Alante,São José dos Campos,2024,Ativos,Supply,Capex,Real,11,carteiras diversas,"[carteiras, diversas]"
1,40423.0,Elite,Idesa Taubaté,2024,Ativos,Supply,Capex,Real,11,1 carteira pne / 100 carteira univer. bm2002,"[carteira, pne, carteira, univer]"
2,15146.0,Elite,Idesa Taubaté,2024,Ativos,Supply,Capex,Real,11,quadro gestão / mural de aviso / quadro 400x120,"[quadro, gestão, mural, aviso, quadro]"
3,21530.0,Anglo Alante,São José dos Campos,2024,Ativos,Supply,Capex,Real,11,projetores epson,"[projetores, epson]"
5,11800.0,Anglo Alante,São José dos Campos,2024,Ativos,Supply,Capex,Real,11,bebedouro,[bebedouro]
...,...,...,...,...,...,...,...,...,...,...,...
17,8560.0,pH,Freguesia,2024,Ativos,Supply,Capex,Real,1,móveis tramontina,"[móveis, tramontina]"
18,6136.0,pH,Freguesia,2024,Ativos,Supply,Capex,Real,1,móveis,[móveis]
19,3699.0,Coleguium,Nova Suíça,2024,Ativos,Supply,Capex,Real,1,caixa de som bluetooth,"[caixa, som, bluetooth]"
20,3699.0,Coleguium,Nova Suíça,2024,Ativos,Supply,Capex,Real,1,caixa de som bluetooth,"[caixa, som, bluetooth]"
